Importação das bibliotecas


In [3]:
import numpy as np
from scipy.stats import norm
import scipy.stats

Dados de produção diária de bicicletas de determinada empresa nos últimos 60 dias

In [4]:
dados = [
    85, 88, 66, 80, 80, 54, 91, 67, 80, 63, 76, 68,
    70, 80, 60, 55, 64, 59, 57, 70, 81, 59, 81, 76,
    74, 91, 72, 54, 60, 78, 59, 76, 70, 60, 79, 71,
    49, 57, 81, 93, 63, 73, 64, 78, 77, 61, 78, 72,
    67, 63, 73, 77, 67, 84, 68, 75, 65, 74, 60, 84
]


Calcular a estatística de teste de Shapiro-Francia

In [5]:
n = len(dados)
data_sorted = sorted(dados)
x_bar = np.mean(dados)

In [ ]:
m = [norm.ppf((i - 0.375) / (n + 0.25)) for i in range(1, n+1)]
w_calc = sum((x - x_bar)**2 for x in data_sorted) / sum((x - m_i)**2 for x, m_i in zip(data_sorted, m))

Obter o valor crítico da tabela H.1


In [7]:
alpha = 0.05
w_critical = scipy.stats.shapiro(dados)[0]

Comparar o resultado e tirar a conclusão

In [ ]:
if w_calc < w_critical:
    print("Rejeita-se a hipótese nula. Os dados não provêm de uma população com distribuição normal.")
else:
    print("Não se rejeita a hipótese nula. Os dados provêm de uma população com distribuição normal.")

Rejeita-se a hipótese nula. Os dados não provêm de uma população com distribuição normal.
